In [39]:
import numpy as np
import matplotlib.pyplot as plt 
from scipy.signal import butter, filtfilt, hilbert, convolve
import mne

raw = mne.io.read_raw_edf('/Users/ohadgonen/Desktop/Neuroscience/Year 3/1st semester/Erez lab/sub-01_ses-01_task-verbalWM_run-01_ieeg.edf', preload=True)
print(raw.info)


Extracting EDF parameters from /Users/ohadgonen/Desktop/Neuroscience/Year 3/1st semester/Erez lab/sub-01_ses-01_task-verbalWM_run-01_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 799999  =      0.000 ...   400.000 secs...
<Info | 8 non-empty values
 bads: []
 ch_names: mAHL1, mAHL2, mAHL3, mAHL4, mAHL5, mAHL6, mAHL7, mAHL8, mAL1, ...
 chs: 48 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 1000.0 Hz
 meas_date: 2023-09-13 10:27:08 UTC
 nchan: 48
 projs: []
 sfreq: 2000.0 Hz
 subject_info: <subject_info | his_id: <no>
>


In [43]:
# Load EDF file
raw = mne.io.read_raw_edf("/Users/ohadgonen/Desktop/Neuroscience/Year 3/1st semester/Erez lab/sub-01_ses-01_task-verbalWM_run-01_ieeg.edf", preload=True)

# === Compute Phase-Locking Value (PLV) on your EDF (MNE) ===


# Frequency of interest and band 

low_band = (12, 30)    # low-frequency band (e.g., beta)
high_band = (70, 250)  # high-frequency band (e.g., high gamma)

trial_duration = 400    # in seconds

# Optional: a "bell" interval to highlight on the plot (purely visual)
bell_window = (0.3, 0.5)   # in seconds within a trial; set to None to disable

# ---------------------------------------------------------
# ==== Step 1: Load EDF and import signals ====
# ---------------------------------------------------------
fs  = float(raw.info["sfreq"])   # 2000 Hz

# Select two channels for PLV computation
ch_name_1 = raw.ch_names[0]
ch_name_2 = raw.ch_names[1]

# Pull channels as 1D arrays (n_samples,)
ch1 = raw.get_data(picks=[4]).squeeze()
ch2 = raw.get_data(picks=[16]).squeeze()

# (optional) Demean to reduce edge/filter artifacts
ch1 = ch1 - np.mean(ch1)
ch2 = ch2 - np.mean(ch2)

 
# ---------------------------------------------------------
# ==== Step 2: Bandpass Filter using Butterworth filter ====
# ---------------------------------------------------------
def bandpass_filter(data, lowcut, highcut, fs, order=4):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return filtfilt(b, a, data)

lf_filtered = bandpass_filter(ch1, low_band[0], low_band[1], fs)
hf_filtered = bandpass_filter(ch2, high_band[0], high_band[1], fs)

# ---------------------------------------------------------
# ==== Step 3: Hilbert transform → instantaneous phase ====
# ---------------------------------------------------------
analytic_lf = hilbert(lf_filtered)
analytic_hf = hilbert(hf_filtered)

phase_lf = np.angle(analytic_lf)  # Instantaneous phase of low-frequency signal
amp_hf = np.abs(analytic_hf)  # Instantaneous amplitude of high-frequency signal

analytic_hf_abs = hilbert(amp_hf)
phase_hf = np.angle(analytic_hf_abs)  # Instantaneous phase of high-frequency amplitude

# ---------------------------------------------------------
# ==== Step 4: Phase difference and PLV ====
# ---------------------------------------------------------
phase_diff = phase_lf - phase_hf   # 1D array (n_samples,)
plv_global = float(np.abs(np.mean(np.exp(1j * phase_diff))))
print(f"Global PLV: {plv_global:.3f}")

Extracting EDF parameters from /Users/ohadgonen/Desktop/Neuroscience/Year 3/1st semester/Erez lab/sub-01_ses-01_task-verbalWM_run-01_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 799999  =      0.000 ...   400.000 secs...
Global PLV: 0.003


In [44]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt, hilbert
import mne

# ---------------------------------------------------------
# ==== Step 1: Load EDF and import signals ====
# ---------------------------------------------------------
raw = mne.io.read_raw_edf(
    "/Users/ohadgonen/Desktop/Neuroscience/Year 3/1st semester/Erez lab/sub-01_ses-01_task-verbalWM_run-01_ieeg.edf",
    preload=True,
)
fs = float(raw.info["sfreq"])  # Sampling frequency (e.g., 2000 Hz)
print(raw.info)

# Frequency bands
low_band = (12, 30)  # Low-frequency band (e.g., beta)
high_band = (70, 250)  # High-frequency band (e.g., high gamma)

# ---------------------------------------------------------
# ==== Step 2: Bandpass Filter using Butterworth filter ====
# ---------------------------------------------------------
def bandpass_filter(data, lowcut, highcut, fs, order=4):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype="band")
    return filtfilt(b, a, data)

# ---------------------------------------------------------
# ==== Step 3: Iterate over all channel pairs ====
# ---------------------------------------------------------
channel_names = raw.ch_names
n_channels = len(channel_names)
highest_plv = 0
best_pair = None

for i in range(n_channels):
    for j in range(i + 1, n_channels):  # Avoid duplicate pairs and self-coupling
        ch1 = raw.get_data(picks=[i]).squeeze()
        ch2 = raw.get_data(picks=[j]).squeeze()

        # (Optional) Demean to reduce edge/filter artifacts
        ch1 = ch1 - np.mean(ch1)
        ch2 = ch2 - np.mean(ch2)

        # Bandpass filter
        lf_filtered = bandpass_filter(ch1, low_band[0], low_band[1], fs)
        hf_filtered = bandpass_filter(ch2, high_band[0], high_band[1], fs)

        # Hilbert transform
        analytic_lf = hilbert(lf_filtered)
        analytic_hf = hilbert(hf_filtered)

        phase_lf = np.angle(analytic_lf)  # Instantaneous phase of low-frequency signal
        amp_hf = np.abs(analytic_hf)  # Instantaneous amplitude of high-frequency signal

        analytic_hf_abs = hilbert(amp_hf)
        phase_hf = np.angle(analytic_hf_abs)  # Instantaneous phase of high-frequency amplitude

        # Compute PLV
        phase_diff = phase_lf - phase_hf
        plv_global = float(np.abs(np.mean(np.exp(1j * phase_diff))))

        # Check if this is the highest PLV so far
        if plv_global > highest_plv:
            highest_plv = plv_global
            best_pair = (channel_names[i], channel_names[j])

# ---------------------------------------------------------
# ==== Step 4: Output the highest PLV and the best pair ====
# ---------------------------------------------------------
print(f"Highest PLV: {highest_plv:.3f}")
print(f"Best channel pair: {best_pair[0]} ↔ {best_pair[1]}")

Extracting EDF parameters from /Users/ohadgonen/Desktop/Neuroscience/Year 3/1st semester/Erez lab/sub-01_ses-01_task-verbalWM_run-01_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 799999  =      0.000 ...   400.000 secs...
<Info | 8 non-empty values
 bads: []
 ch_names: mAHL1, mAHL2, mAHL3, mAHL4, mAHL5, mAHL6, mAHL7, mAHL8, mAL1, ...
 chs: 48 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 1000.0 Hz
 meas_date: 2023-09-13 10:27:08 UTC
 nchan: 48
 projs: []
 sfreq: 2000.0 Hz
 subject_info: <subject_info | his_id: <no>
>
Highest PLV: 0.011
Best channel pair: mAHL6 ↔ mAHL7


In [ ]:
# Glossary of channel names to indices

# Get all channel names
channels = raw.ch_names

# Find their indices
i_ahl5 = channels.index('mAHL5')
i_ecl1 = channels.index('mECL1')

print(f"mAHL5 index: {i_ahl5}")
print(f"mECL1 index: {i_ecl1}")

mAHL5 index: 4
mECL1 index: 16
